In [4]:
print('hello world')

hello world


In [1]:
import sys
from pathlib import Path

def get_project_root():
    for path in (Path.cwd(), *Path.cwd().parents):
        if (path / 'src').exists():
            return path
    raise ModuleNotFoundError("Could not find 'src' directory from current working directory.")

project_root = get_project_root()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.plot_data_utils import PlotDataUtils
from src.classical_MILP_solver import ClassicalMILPSolver

In [2]:
# Initialise utilities for the input CSV data
data_utils = PlotDataUtils(Path('../data/input_data.csv'))
df = data_utils.load_data()

df = data_utils.compute_wind_stats(df)
wind_rev_by_s, wind_rev_exp, wind_rev_min, wind_rev_max, wind_rev_var = data_utils.wind_revenue(df)


In [3]:
def solve_qaoa_for_dv(initial_qaoa_angles, reps):

    fake_backend = FakeManilaV2() 
    mock_backend = AerSimulator.from_backend(fake_backend)
    
    qubo_program = QuadraticProgram("Battery_Binary_Subproblem")

    for t in hours:
        qubo_program.binary_var(name=f'x_{t}')
        
    PENALTY_FACTOR = 100
    linear_coeffs = {}
    for t in hours:
        linear_coeffs[f'x_{t}'] = prices[t] * (power_charge_max + power_discharge_max)

        if t == hours[-1]: 
            # If x_24=1, cost increases by massive penalty.
            linear_coeffs[f'x_{t}'] += PENALTY_FACTOR 
            
        if t == hours[0]: 
            # If x_1=0, cost is lower, but we push it toward x_1=1 by adding a large negative term.
            linear_coeffs[f'x_{t}'] -= PENALTY_FACTOR 
            
    qubo_program.minimize(linear=linear_coeffs)

    # Convert the QUBO to an Ising Operator (Hamiltonian)
    qubo_converter = QuadraticProgramToQubo()
    qubo = qubo_converter.convert(qubo_program)
    operator, offset = qubo.to_ising()   ### H_c


    # 1. Initialize the custom mixer operator as an empty operator
    num_qubits = len(hours)
    mixer_operator = SparsePauliOp("I" * num_qubits, coeffs=[0.0]) # Initialize to zero

    # 2. Loop through all adjacent pairs (t and t+1)
    for i in range(num_qubits - 1):
    
        # Pauli string for the X_t * X_{t+1} term
        # Creates an operator like: I I ... X X I ... I
        pauli_XX = ['I'] * num_qubits
        pauli_XX[i] = 'X'      # X on qubit t
        pauli_XX[i+1] = 'X'    # X on qubit t+1
    
        # Pauli string for the Y_t * Y_{t+1} term
        pauli_YY = ['I'] * num_qubits
        pauli_YY[i] = 'Y'      # Y on qubit t
        pauli_YY[i+1] = 'Y'    # Y on qubit t+1
    
        # The coefficient for each term is 1/2
        coeff = 0.5
    
        # Add the terms to the mixer operator
        # Note: SparsePauliOp takes the Pauli string as a list of characters
        mixer_operator += SparsePauliOp("".join(pauli_XX[::-1]), coeffs=[coeff])
        mixer_operator += SparsePauliOp("".join(pauli_YY[::-1]), coeffs=[coeff])
    
    
    # 2.1 Define the Ansatz (Circuit) 
    ansatz = QAOAAnsatz(cost_operator=operator, reps=reps, mixer_operator=mixer_operator)
    
    # 2.2 Define the Estimator and Cost Function
    estimator = Estimator() 
    
    def cost_func(params):
        # Calculates the expectation value <psi|H|psi> (Energy/Cost)
        job = estimator.run([ansatz], [operator], parameter_values=[params], backend=mock_backend) 
        result = job.result()
        
        energy = result.values[0]
        return energy.real

    # 2.3 Optimize (Classical Step)
    initial_point = initial_qaoa_angles
    
    # Minimize the cost function (Energy) using COBYLA
    res = minimize(cost_func, initial_point, method='COBYLA', options={'maxiter': 10}) 
    
    # --- 3. Extract Result using Sampler (Inference) ---
    sampler = Sampler() 
    
    # Bind optimal parameters to the circuit and add measurements
    final_bound_circuit = ansatz.assign_parameters(res.x)
    final_bound_circuit.measure_all()
    
    # Sample the optimized circuit 
    final_job = sampler.run([final_bound_circuit], backend=mock_backend)
    final_result = final_job.result()
    
    # Get the key (bitstring/integer) with the maximum quasi-probability
    quasi_dists = final_result.quasi_dists[0]
    best_bit_key = max(quasi_dists, key=quasi_dists.get) 
    
    # Convert key to a binary string representation
    num_qubits = len(hours) 
    best_bit_string = format(best_bit_key, '0' + str(num_qubits) + 'b')
    
    # Final Conversion: Reverse the bitstring to correct for Qiskit's endianness
    optimal_is_charge = [int(bit) for bit in best_bit_string[::-1]]

    return optimal_is_charge, res.x

In [4]:
# =================================
# --- PARAMETERS ---
# =================================

# Extract Wind farm parameters
hours = df['hour'].astype(int).tolist()
prices = dict(zip(df['hour'], df['price']))

# Battery parameters
power_charge_max = 5.0   # MW
power_discharge_max = 4.0  # MW
energy_cap = 16.0        # MWh
eta_c = 0.8              # charging efficiency
eta_d = 1.0              # discharging efficiency
max_cycles = 2           # full cycles/day cap

# Constant term: expected wind revenue across scenarios (equiprobable)
expected_wind_revenue = (df['mean_wind'] * df['price']).sum()


In [7]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import pulp
from scipy.optimize import minimize 
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit_aer.primitives import Estimator, Sampler
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp 
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.problems import QuadraticProgram 
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

initial_dv=[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
num_deph=2
initial_qaoa_angles = 2 * np.pi * np.random.rand(2 * num_deph)


def output_bitstring_QAOA(initial_dv,initial_qaoa_angles,num_deph):

    current_dv_solution = initial_dv
    current_qaoa_angles = 2 * np.pi * np.random.rand(2 * num_deph) 
  
    qaoa_angles = current_qaoa_angles 
    
    new_dv_solution, new_optimal_angles = solve_qaoa_for_dv(
        qaoa_angles,  #### QAOA angles 
        num_deph  #### QAOA circuit depth
        )

    current_dv_solution = new_dv_solution
    current_dv_solution[0]=0 ## impose boundary conditions x_0=0
    current_dv_solution[-1]=0 ## impose boundary conditions x_24=0
    current_qaoa_angles = new_optimal_angles 
    

    return current_dv_solution



In [8]:
charge_pattern=output_bitstring_QAOA(initial_dv,initial_qaoa_angles,num_deph)

/var/folders/ls/f9tjq1sn7rn0gw726qrzmy580000gn/T/ipykernel_73958/275218029.py:30: DeprecationWarning: Estimator has been deprecated as of Aer 0.15, please use EstimatorV2 instead.
  new_dv_solution, new_optimal_angles = solve_qaoa_for_dv(
/var/folders/ls/f9tjq1sn7rn0gw726qrzmy580000gn/T/ipykernel_73958/275218029.py:30: DeprecationWarning: Option approximation=False is deprecated as of qiskit-aer 0.13. It will be removed no earlier than 3 months after the release date. Instead, use BackendEstimator from qiskit.primitives.
  new_dv_solution, new_optimal_angles = solve_qaoa_for_dv(
/var/folders/ls/f9tjq1sn7rn0gw726qrzmy580000gn/T/ipykernel_73958/275218029.py:30: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  new_dv_solution, new_optimal_angles = solve_qaoa_for_dv(


In [9]:
charge_pattern

[0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0]

In [ ]:
# 24-length 0/1 pattern: 1 => charging, 0 => not charging
#charge_pattern = [0]*8 + [1]*4 + [0]*12

solver = ClassicalMILPSolver(charge_pattern=charge_pattern, lambda_switch=00.0)

schedule, status, battery_profit, total_revenue = solver.solve(df, wind_rev_exp)


PulpSolverError: PULP_CBC_CMD: Not Available (check permissions on cbc)

In [ ]:
print(f"Solver status: {status}")
print(f"Battery profit: EUR {battery_profit:,.2f}")
print(f"Total expected revenue (wind + battery): EUR {total_revenue:,.2f}")
print(schedule.head())

In [ ]:
solver.plot_battery_schedule(schedule)